In [18]:
import json
import os 
import pandas as pd


In [2]:
carpeta = '../../dataRAW/'

lista_combinada = []

for archivo in os.listdir(carpeta):
    if archivo.endswith('.json'):
        ruta_archivo = os.path.join(carpeta, archivo)  
        with open(ruta_archivo, 'r') as f:
            datos = json.load(f)
            if isinstance(datos, list): 
                lista_combinada.extend(datos) 

print(len(lista_combinada))

196917


In [3]:
ids = []
ids_vistos = set()

for x in lista_combinada:
    _id = x["id"]
    if _id not in ids_vistos:
        ids.append(_id)  # Añadir a la lista manteniendo el orden
        ids_vistos.add(_id)  # Añadir al conjunto para evitar duplicados

In [4]:
def remove_duplicates(ids, list):
    ret = []
    i = 0
    for id in ids:
        while list[i]["id"] != id:
            i = i+1
        ret.append(list[i])
    return ret

data = remove_duplicates(ids, lista_combinada)
data = [x for x in data if isinstance(x, dict)]


In [5]:
print(len(data))

172111


In [47]:
'''
bodyType None = Undetermined (can be anyone)
bodyType 1 = Berlina
bodyType 2 = Coupe
bodyType 3 = Cabrio
bodyType 4 = Familiar
bodyType 5 = Monovolumen
bodyType 6 = SUV
bodyType 7 = Pick Up
bodyType 8 = Furgoneta
'''
def assign_body_type(car):
    try:

        if not isinstance(car, dict): return None

        if isinstance(car["bodyTypeId"], str): return car["bodyTypeId"]

        typeID = car["bodyTypeId"]
        match typeID:
            case None: return None
            case 1: return "Berlina"
            case 2: return "Coupe"
            case 3: return "Cabrio"
            case 4: return "Familiar" 
            case 5: return "Monovolumen"
            case 6: return "SUV"
            case 7: return "Pick Up"
            case 8: return "Furgoneta"
        return None
    
    except KeyError:
        return None

def extraer_marca(texto):
    palabras = texto.split()  # Dividir el string por espacios
    return palabras[0]

def get_images(car):
    if not isinstance(car, dict): return None
    try:
        ret = []
        for resource in car["resources"]:
            if resource['type'] == 'IMAGE' : ret.append(resource['url'])
        return ret
    except KeyError as e:
        print("uwu la clave no existe")
        return None

In [48]:
ids = [x["id"] for x in data]
models = [assign_body_type(x) for x in data]
brand = [extraer_marca(x["title"]) for x in data]
images = [get_images(x) for x in data]

In [49]:
dataframe = pd.DataFrame({"id": ids, "model": models, "brand": brand, "images": images})

In [50]:
dataframe

,id,model,brand,images
0,58841374,Furgoneta,DACIA,[https://a.ccdn.es/cnet/2024/10/01/58841374/19...
1,58638307,SUV,MITSUBISHI,[https://a.ccdn.es/cnet/2024/09/04/58638307/19...
2,58798715,None,MERCEDES-BENZ,[https://a.ccdn.es/cnet/2024/09/25/58798715/19...
3,58887285,None,RENAULT,[https://a.ccdn.es/cnet/2024/10/06/58887285/19...
4,58765577,Furgoneta,RENAULT,[https://a.ccdn.es/cnet/2024/09/20/58765577/19...
...,...,...,...,...
172106,58788502,Berlina,FORD,[https://a.ccdn.es/cnet/vehicles/16407331/2b65...
172107,58600883,Berlina,FORD,[https://a.ccdn.es/cnet/vehicles/16259881/e479...
172108,58780688,Berlina,VOLKSWAGEN,[https://a.ccdn.es/cnet/vehicles/16400998/5ebb...
172109,58708761,Berlina,RENAULT,[https://a.ccdn.es/cnet/2024/09/13/58708761/19...


In [54]:
print(len(dataframe))
dataframe.dropna(inplace=True)
print(len(dataframe))

172111
152602


In [55]:
max_file_size = 100 * 1024 * 1024

def split_dataframe_to_csv(df, max_file_size, output_folder ,base_filename):
    file_index = 1
    chunk_size = len(df) // 7 
    while chunk_size > 0:
        start_row = 0
        while start_row < len(df):
            end_row = start_row + chunk_size
            chunk = df.iloc[start_row:end_row]
            filename = f"{output_folder}/{base_filename}_part{file_index}.csv"
            chunk.to_csv(filename, index=False)
            if os.path.getsize(filename) > max_file_size:
                chunk_size //= 2
                os.remove(filename)
            else:
                start_row = end_row
                file_index += 1

        if start_row >= len(df):
            break

split_dataframe_to_csv(dataframe, max_file_size, "photos_urls/", "photos")